In [1]:
from qadence2.expressions import *

In [2]:
a = parameter("a")
b = parameter("b")
phi = variable("phi")

In [3]:
a * (cos(phi / 2) * X(1) - sin(phi / 2) * Y(1)) - b * Z(1)

a cos(0.5 phi) X[1] - a sin(0.5 phi) Y[1] - b Z[1]

In [4]:
X(3) * Y(1) * Z(2)

Y[1] Z[2] X[3]

In [5]:
X(target=(1,), control=(0,2))

X[1; 0 2]

In [6]:
n = lambda l: (1 - Z(l)) / 2

n(1) * n(2) + n(2) * n(3)

0.5 - 0.5 Z[2] - 0.25 Z[1] + 0.25 Z[1] Z[2] - 0.25 Z[3] + 0.25 Z[2] Z[3]

In [7]:
CZ = unitary_hermitian_operator('CZ')

expr = RX(a * phi / 2)(2) * CZ() * RY(b * phi / 2)(0)

irc(expr)

Model(
  register=AllocQubits(
    num_qubits=3,
    qubit_positions=[],
    grid_type=None,
    grid_scale=1.0,
    options={},
  ),
  inputs={
    'a': Alloc(1, trainable=False),
    'phi': Alloc(1, trainable=True),
    'b': Alloc(1, trainable=False),
  },
  instructions=[
    Assign('%0', Call('mul', 0.5, Load('a'))),
    Assign('%1', Call('mul', Load('%0'), Load('phi'))),
    QuInstruct('rx', Support(target=(2,)), Load('%1')),
    QuInstruct('cz', Support.target_all(), attrs={'is_hermitian': True, 'is_unitary': True}),
    Assign('%2', Call('mul', 0.5, Load('b'))),
    Assign('%3', Call('mul', Load('%2'), Load('phi'))),
    QuInstruct('ry', Support(target=(0,)), Load('%3')),
  ],
  directives={},
  settings={},
)

In [8]:
set_qubits_positions([(-1,0), (-1, 1), (1, 0), (1, 1)])
set_grid_type("triangular")

t = time_variable("t")
omega = array_variable("omega", 4)
detuning = array_variable("detuning", 3)
phase = parameter("phase")

expr = (
    NativeDriven(t / 2, omega, detuning, phase)()
    * FreeEvolution(2.5)()
    * NativeDriven(t / 2, omega, -detuning, phase)()
)

irc(expr)

Model(
  register=AllocQubits(
    num_qubits=4,
    qubit_positions=[(-1, 0), (-1, 1), (1, 0), (1, 1)],
    grid_type=triangular,
    grid_scale=1.0,
    options={},
  ),
  inputs={
    't': Alloc(1, trainable=True, attrs={'time_parameter': True}),
    'omega': Alloc(4, trainable=True),
    'detuning': Alloc(3, trainable=True),
    'phase': Alloc(1, trainable=False),
  },
  instructions=[
    Assign('%0', Call('mul', 0.5, Load('t'))),
    QuInstruct('dyn_pulse', Support.target_all(), Load('%0'), Load('omega'), Load('detuning'), Load('phase')),
    QuInstruct('dyn_wait', Support.target_all(), 2.5),
    Assign('%1', Call('mul', -1.0, Load('detuning'))),
    QuInstruct('dyn_pulse', Support.target_all(), Load('%0'), Load('omega'), Load('%1'), Load('phase')),
  ],
  directives={},
  settings={},
)